<img src="https://assets.ensam.eu/logo/fr/logo-trans-322x84.png" style="width:256px" > <img src="https://upload.wikimedia.org/wikipedia/commons/1/12/Cc-by-nc-sa_icon.svg" width=192 align=right>

**Author :** Jean-Christophe Loiseau

**Email :** [jean-christophe.loiseau@ensam.eu](mailto:jean-christophe.loiseau@ensam.eu)

**Date :** June 2025

---

In [1]:
#> Standard Python packages.
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

#> PySINDy
import pysindy as ps

/home/loiseau/miniconda3/envs/cea_saclay/lib/python3.12/site-packages/pysindy/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound


# **Nonlinear system identification with PySINDy**

**NOTE :** This notebook is an introduction to the python package `PySINDy`. It is primarily adapted from the example notebooks found in the `PySINDy` Github repo [(here)](https://github.com/dynamicslab/pysindy/tree/master/examples).

### **Acknowledgements**

This work was made possible thanks to the financial support of the French National Agency for Research (ANR) through the ANR-33-CE46-0008-CONMAN grant agreement.

<img src="https://upload.wikimedia.org/wikipedia/fr/c/c3/Nouveau_logo_ANR_2022.jpg" style="width:256px" >